## Select Environment & Start Date

In [ ]:
env = "PAPER" # PAPER / PROD / BACKTEST
start_date = "2020-10-01"

#### Imports

In [ ]:
# silence warnings
import warnings
import liualgotrader.analytics.analysis as ana
import matplotlib.pyplot as plt
import nest_asyncio
import numpy as np
import pandas as pd
import pyfolio as pf
from empyrical import roll_max_drawdown
from scipy.stats import kurtosis, skew

%matplotlib inline
warnings.filterwarnings("ignore")


nest_asyncio.apply()

#### Load Portfolio data from database

In [ ]:
returns, investments, percentages = ana.portfolio_return(env, start_date)

## Show Portfolio analysis
### Revenue & Percentage per strategy

In [ ]:
columns = list(returns.columns)
columns.insert(0, columns.pop(columns.index("revenue")))
with plt.xkcd():
    for column in columns:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))  # 1 row, 2 columns
        ax1.set_ylabel(f"{column} $")
        ax2.set_ylabel(f"{column} %")
        plt.xticks(rotation=45)
        returns[column].plot(ax=ax1)
        percentages[column].plot(ax=ax2, color="red")
        plt.tight_layout()

### Accumulative Revenue & Draw-down

In [ ]:
columns = list(returns.columns)
columns.insert(0, columns.pop(columns.index("revenue")))
for column in columns:
    fig, (ax1, ax2) = plt.subplots(
        2, 1, figsize=(10, 4), gridspec_kw={"height_ratios": [3, 1]}
    )  
    ax1.set_ylabel(f"Accumulative\n{column} $")
    plt.xticks(rotation=45)
    returns[column].cumsum().plot(ax=ax1)
    roll_max_drawdown(returns[column].cumsum(), window=1).plot(ax=ax2)
    ax2.set_ylabel("drawdown")
    plt.tight_layout()

### Strategy performance distribution
#### Summary

In [ ]:
df = pd.DataFrame()
df["strategy"] = returns.columns.unique()
df["mean"] = df.strategy.apply(lambda x: returns[x].dropna().mean())
df["std"] = df.strategy.apply(lambda x: returns[x].dropna().std())
df["skew"] = df.strategy.apply(lambda x: skew(returns[x].dropna()))
df["kurtosis"] = df.strategy.apply(lambda x: kurtosis(returns[x].dropna()))
df

In [ ]:
columns = list(returns.columns)
columns.insert(0, columns.pop(columns.index("revenue")))
with plt.xkcd():
    for column in columns:
        returns[column].hist()
        plt.xticks(rotation=45)
        plt.ylabel(column)
        plt.show()